# American Options Pricing Analytics
------------------
> **Idriss Afra**

This project aims to price American options using the Binomial model, the Barone-Adesi & Whaley approximation, and the Least-Squares MC method.

## The Binomial Model

The Binomial model is relatively easy to understand and implement as it assumes that the underlying asset $S_t$ moves up or down by a fixed percentage in each period.

Let's define the movement factors $u$ and $d$ as following :

$$
u = e^{\sigma\sqrt{\Delta t}}\qquad d = e^{-\sigma\sqrt{\Delta t}} \\
$$
Where $\sigma$ is the volatility and $Δt$ is a tiny time step to maturity $T$. The risk-neutral probability under this model is :

$$
p = \frac{e^{(r-q)\Delta t}-d}{u-d} \\
$$
So that :
$$
E(S_{t+\Delta t}) = p × u × S_t + (1 - p) × d × S_t  
= p × S_{t+\Delta t}^u + (1 - p) × S_{t+\Delta t}^d \\
$$

Where $r$ is the zero-coupon interest rate and $q$ the dividend yield.

Once the Binomial tree is simulated, the options are priced using a backward method :

* European Style : 

$$
PV_t(S_t) = e^{-r\Delta t}\left(p\times PV_{t+\Delta t}(S_{t+\Delta t}^u)+(1-p)\times PV_{t+\Delta t}(S_{t+\Delta t}^d)\right)
$$

* American Style :    

$$
PV_t(S_t) = max\left(S_t - K, e^{-r\Delta t}\left(p\times PV_{t+\Delta t}(S_{t+\Delta t}^u)+(1-p)\times PV_{t+\Delta t}(S_{t+\Delta t}^d)\right)\right) \\
$$

With the final condition being : $ PV_T(S) = Max\left(ϕ × (S - K), 0 \right)  $

Where $ϕ = 1$ for call options and $ϕ = -1$ for put options, and $K$ is the strike price.

The accuracy of the binomial model improves with the number of time steps. However, this also raises the complexity and computation time.

In [1]:
import numpy as np
import math

class binomial_model:
    """
    The Binomial model class.
    """
    def __init__(self, n_steps=10**4):
        """
        Init method.
        n_steps is the number of time steps.
        """
        self.n_steps = n_steps
        self.set_data = False
    
    def set_option_data(self, S0, T, vol, r, q=0):
        """
        Setter of option data.
        S0 : spot price
        T : maturity
        vol : ATM implied volatility
        r : zero-coupon interest rate
        q : dividend yield
        """
        self.S0 = S0
        self.T = T
        self.vol = vol
        self.r = r
        self.q = q
        self.set_data = True
    
    def spot_simulation(self):
        """
        The spot simulation under the Binomial model.
        The function returns the Binomial tree.
        """
        if self.set_data == False :
            print("Please set option data first.")
            return None

        dt = self.T / self.n_steps
        u = math.exp(self.vol * math.sqrt(dt))
        d = 1. / u

        # Spot simulation : Binomial tree
        s = np.zeros((self.n_steps+1, self.n_steps+1))
        s[0,0] = self.S0
        for i in range(1, self.n_steps+1) :
            s[:i,i] = s[:i,i-1] * u
            s[i,i] = s[i-1,i-1] * d

        return s
    
    def binomial_price(self, isCall, K, amer=True):
        """
        The Binomial price.
        """
        if self.set_data == False :
            print("Please set option data first.")
            return None

        # Spot simulation : Binomial tree
        dt = self.T / self.n_steps
        u = math.exp(self.vol * math.sqrt(dt))
        d = 1. / u
        p = (math.exp((self.r - self.q) * dt) - d) / (u - d)
        s = np.zeros((self.n_steps+1, self.n_steps+1))
        s[0,0] = self.S0
        for i in range(1, self.n_steps+1) :
            s[:i,i] = s[:i,i-1] * u
            s[i,i] = s[i-1,i-1] * d

        # Option payoff at maturity
        phi = 1 if isCall else -1
        v = np.maximum(phi * (s[:, self.n_steps] - K), 0.)

        # Discount between 2 time steps
        discount = math.exp(-self.r * dt)

        # Backward loop
        for i in range(self.n_steps - 1, -1, -1) : # => i = n_steps-1 ... 0
            n_nodes = i + 1 # i+1 nodes at time step #i
            v = discount * (p * v[:n_nodes] + (1 - p) * v[1:n_nodes+1])
            if amer :
                # Possibility of early exercise
                v = np.maximum(phi * (s[:n_nodes, i] - K), v)

        return v[0]

## The Barone-Adesi & Whaley Model

The Barone-Adesi & Whaley (BAW) model is an extension of the Black-Scholes model designed to price American options. It works by estimating the optimal early-exercise point, also called the exercise frontier, and approximating a premium to account for the early exercise.
<br>While the BAW model offers a good balance between speed and accuracy, it tends to be less accurate for options with medium maturities, but performs better for short- and long-term options.

We define the constants : $M = \frac{2r}{σ^2}$, and $N = \frac{2(r-q)}{σ^2}$.

The BAW price of an American option with strike $K$ and maturity $T$ is :

\begin{equation}
\begin{split}
PV_{B.A.W}^{Amr}(K, T) & = PV_{Black}^{Eur}(K, T) + A × \left( \frac{S}{S^*} \right)^b;\space ϕ ×  (S - S^*) < 0 \\
PV_{B.A.W}^{Amr}(K, T) & = ϕ \times (S - K);\space ϕ ×  (S - S^*) \ge 0
\end{split}
\end{equation}

Where :    
- $A = ϕ \times \left( \frac{S^*}{q} \right) × \left( 1 - e^{-qT} N_{(0, 1)} \left(ϕ \times d_1(S^*) \right) \right)$


- $d_1(S) = \frac{log\left(\frac{S}{K}\right) + \left(r - q +  \frac{σ\sqrt T}{2}\right)}{σ\sqrt T}$


- $b = 0.5 × \left(1 - N + ϕ × \sqrt{(1 - N)^2 + 4M/K}  \right)$


- $S^*$ represent the "Optimal Spot" (Exercise Frontier) and is the solution of the following equation :

$$
ϕ × (S - K) = PV_{Black}^{Eur}(K, T) + ϕ × \left( 1 - e^{-qT} N_{(0, 1)} \left(ϕ \times d_1(S) \right) \right) × S / q
$$

Solving the $S^*$ requires numerical methods as its equation has no closed-form solution. In this project, we combine both the Newton-Raphson and the Bisection methods.

More details can be found in [Barone-Adesi-Whaley 1987](https://github.com/Idriss-Afra/American-Options-Pricing-Analytics/blob/main/Barone-Adesi-Whaley%201987.pdf).

In [2]:
from scipy.stats import norm
from scipy import optimize
import sys

class baw_model :
    """
    The Barone-Adhesi & Whaley model class.
    """
    def __init__(self):
        """
        Init method.
        """
        self.set_data = False

    def set_option_data(self, S0, T, vol, r, q=0):
        """
        Set option data method
        S0 : spot price
        T : maturity
        vol : ATM implied volatility
        r : zero-coupon interest rate
        q : dividend yield
        """
        self.S0 = S0
        self.T = T
        self.vol = vol
        self.r = r
        self.q = q
        self.set_data = True

    def d1(self, S, K):
        """
        The Black-Scholes d1 formula.
        """
        if self.set_data == False :
            print("Please set option data first.")
            return None

        v2T = self.vol**2 * self.T
        return (np.log(S / K) + (self.r - self.q) * self.T + v2T / 2) / v2T**0.5

    def black_price(self, iCall, S, K):
        """
        The Black-Scholes price.
        """
        if self.set_data == False :
            print("Please set option data first.")
            return None

        v2T = self.vol**2 * self.T
        d_1 = self.d1(S, K)
        d_2 = d_1 - v2T**0.5
        phi = 1 if iCall else  -1
        return phi * (S * math.exp(-self.q * self.T) * norm.cdf(phi * d_1) - K * math.exp(-self.r * self.T) * norm.cdf(phi * d_2))

    def _abs_black_delta(self, iCall, S, K):
        """
        The Black-Scholes absolute delta.
        /!\ For internal use only /!\
        """
        if self.set_data == False :
            print("Please set option data first.")
            return None

        phi = 1 if iCall else  -1
        return math.exp(-self.q * self.T) * norm.cdf(phi * self.d1(S, K))

    def _american_adj(self, iCall, S, S_optimal, K):
        """
        The American adjustment formula such as : American_Black_Price = European_Black_Price + American_Adj.
        /!\ For internal use only /!\
        """
        if self.set_data == False :
            print("Please set option data first.")
            return None

        phi = 1 if iCall else  -1
        M = 2 * self.r / self.vol**2
        N = 2 * (self.r - self.q) / self.vol**2
        a = 1 - math.exp(-self.r * self.T)
        b = 0.5 * (1 - N + phi * math.sqrt((1 - N)**2 + 4 * M / a))
        return phi * (S_optimal / b) * (1 - self._abs_black_delta(iCall, S_optimal, K)) * (S / S_optimal)**b

    def optimal_spot(self, iCall, K, n_max=750, n_stdev=3) :
        """
        Newton-Raphson or Bisection method to find the Optimal Spot S*.
        """
        # Computation of S_min and S_max
        Fwd = self.S0 * math.exp((self.r - self.q) * self.T)
        S_min = Fwd / math.exp(n_stdev * self.vol * math.sqrt(self.T))
        S_max = Fwd * math.exp(n_stdev * self.vol * math.sqrt(self.T))
        
        # Choose the adequate numerical algorithm : Newton-Raphson or Bisection
        if self._obj_func(iCall, S_min,  K) * self._obj_func(iCall, S_max,  K) > 0:
            # Newton-Raphson
            init_guess = self.S0
            func = lambda S : self._obj_func(iCall, S,  K)
            func_deriv = lambda S : self._obj_func_deriv(iCall, S,  K)
            return optimize.newton(func, init_guess, func_deriv, maxiter=n_max)
        
        # Bisection
        n = 1
        while n <= n_max :
            optimal_spot = (S_min + S_max)  / 2
            func = self._obj_func(iCall, optimal_spot,  K)
            if (func == 0) or (abs(S_max - S_min) < 1e-6) :
                return optimal_spot
            if func < 0 :
                S_min = optimal_spot
            else :
                S_max = optimal_spot
            n += 1

        return sys.exit("The American volatility calibration algorithm failed to converge. Please review data @ Strike " + str(K))

    def _obj_func(self, iCall, S, K):
        """
        The Optimal Spot S* Equation.
        /!\ For internal use only /!\
        """
        if self.set_data == False :
            print("Please set option data first.")
            return None

        phi = 1 if iCall else  -1
        return self.black_price(iCall, S, K) + self._american_adj(iCall, S, S, K) - phi * (S - K)

    def _obj_func_deriv(self, iCall, S, K):
        """
        The derivative of the Optimal Spot S* Equation.
        /!\ For internal use only /!\
        """
        if self.set_data == False :
            print("Please set option data first.")
            return None

        phi = 1 if iCall else  -1
        M = 2 * self.r / self.vol**2
        N = 2 * (self.r - self.q) / self.vol**2
        a = 1 - math.exp(-self.r * self.T)
        b = 0.5 * (1 - N + phi * math.sqrt((1 - N)**2 + 4 * M / a))
        delta = phi * self._abs_black_delta(iCall, S, K)

        return delta * (1 - 1 / b) + phi * (1 / b) * (1 - phi * math.exp(-self.q * self.T) * norm.pdf(phi * self.d1(S, K)) / (self.vol * math.sqrt(self.T))) - phi

    def baw_price(self, iCall, K):
        """
        The B.A.W price.
        """
        if self.set_data == False :
            print("Please set option data first.")
            return None

        if (iCall and self.r > 0 and self.q == 0):
            # If the underlying does not pay dividends, then the early exercise of a call option in no more advantageous
            return self.black_price(iCall, self.S0, K)
        else :
            phi = 1 if iCall else  -1
            S_optimal = self.optimal_spot(iCall, K)
            #print("The Optimal Spot Price : ", round(S_optimal, 2))
            exercise_cond = phi * (self.S0 - S_optimal) >= 0
            return phi * (self.S0 - K) if exercise_cond else self.black_price(iCall, self.S0, K) + self._american_adj(iCall, self.S0, S_optimal, K)

## The Least-Squares Monte-Carlo Method

The Least-Squares Monte-Carlo method was designed by Longstaff & Schwartz. It is a very interesting and a brilliant pricing method for American and Bermudan payoffs. However, it relies on heavy numerical algorithms which increases its instability and complexity.

First, the spot prices are simulated $M$ times via its Black-Scholes solution :    

$$
S_{t+dt} = S_{t} \times exp\left(\left(r - \frac{\sigma^2}{2}\right) dt + \sigma dW_t \right)
$$

Where : $dt=\frac{T}{N}$ with $N$ being the number of time steps, and $(W_t)_t$ a serie of standard brownian motions.

Then, the American options are priced using a backward loop using the following steps :

1 - We perform a regression method between spot prices simulated at step $t$ and discounted option prices computed at step $t + dt$. In this project, we use a $19$ degree, $20$ coefficients, polynomial regression for $M=6×10^5$ and compute the coefficients $(a_i)_i$ such as :

$$
∀j∈[1,\space 60000], \space  DF_{t,\space t+dt} × PV_{t+dt}(S^{j}_{t+dt}) = ∑_{i=0}^{19}a_i× \left(S^{j}_{t}\right)^i
$$

2 - Then, we compute the continuation price using the solved coefficients :

$$
 C_{t}\left(S^{j}_{t}\right) = ∑_{i=0}^{19}a_i×\left(S^{j}_{t}\right)^i
$$

3 - Finally, we compute the option prices at step $t$ considering the possibility of early exercise :

\begin{equation}
\begin{split}
PV_{t}\left(S^{j}_{t}\right) & = DF_{t,\space t+dt} × PV_{t+dt}\left(S^{j}_{t+dt}\right); \space  C_{t}\left(S^{j}_{t}\right) > ϕ × \left(S^{j}_{t} - K\right) \\ \\
& = ϕ × \left(S^{j}_{t} - K\right);\space C_{t}\left(S^{j}_{t}\right) \le ϕ × \left(S^{j}_{t} - K\right)
\end{split}
\end{equation}

With the final condition being : $ PV_T(S) = Max\left(ϕ × (S - K), 0 \right) $

More details can be found in [Longstaff-Schwartz Least-Squares MC Approach](https://github.com/Idriss-Afra/American-Options-Pricing-Analytics/blob/main/Longstaff-Schwartz%20Least-Squares%20MC%20Approach.pdf).






In [3]:
import warnings
warnings.simplefilter('ignore', np.RankWarning)

# Set the rnd seed @ 0
np.random.seed(0)

class ls_mc :
    """
    The Least-Squares Monte-Carlo Method For American Options.
    """
    def __init__(self, n_simul=6*10**5, n_steps=100, reg_order =19):
        """
        Init method.
        n_simul : number of simulations
        n_steps : number of time steps
        reg_order : order of the polynomial regression
        """
        self.n_simul = n_simul
        self.n_steps = n_steps
        self.reg_order = reg_order
        self.set_data = False

    def set_option_data(self, S0, T, vol, r, q=0):
        """
        Setter of option data.
        S0 : spot price
        T : maturity
        vol : ATM implied volatility
        r : zero-coupon interest rate
        q : dividend yield
        """
        self.S0 = S0
        self.T = T
        self.vol = vol
        self.r = r
        self.q = q
        self.set_data = True

    def set_path(self) :
        """
        Monte-Carlo Simulation under the Black-Scholes model : n_steps x n_simul.
        """
        if self.set_data == False :
            print("Please set option data first.")
            return None
        
        # Generate random N(0,1) (Nb of time steps x Nb of simulations)
        normal = np.random.normal(0, 1, (self.n_steps, self.n_simul))

        # Time steps vector
        self.dt = self.T / self.n_steps
        t = np.linspace(0, self.T, num=self.n_steps+1, endpoint=True)
        sqrt_dt = math.sqrt(self.dt)

        # S array (Nb of time steps x Nb of simulations)
        self.S = np.empty(shape=(self.n_steps + 1, self.n_simul))
        self.S[0,:] = self.S0

        # The drift mu : r - q
        mu = self.r - self.q
        # Spot simulations from t to t + dt
        for j in range(self.n_steps) :
            self.S[j+1,:] = self.S[j,:] * np.exp((mu - 0.5 * self.vol**2) * self.dt + self.vol * sqrt_dt * normal[j,:])

    def mc_price(self, iCall, K) :
        """
        The Monte-Carlo American Price.
        """
        if self.set_data == False :
            print("Please set option data first : \"set_option_data\" method.")
            return None

        if not hasattr(self, 'S') :
            print("Please run spot simulations first : \"set_path\" method")
            return None

        # Option payoff at maturity
        phi = 1 if iCall else -1
        exercise_prices = np.maximum(phi * (self.S - K), 0)
        df = math.exp(-self.r * self.dt)

        # Backward loop
        prices = np.empty(shape=(self.n_steps + 1, self.n_simul))
        prices[-1,:] = exercise_prices[-1,:]
        for j in range(self.n_steps - 1, 0, -1):
            # Polynomial Regression (19 degree - 20 coefficients)
            regression = np.polyfit(self.S[j, :], df * prices[j + 1,:], self.reg_order)
            # Continuation Value
            continuation_prices = np.polyval(regression, self.S[j, :])
            # Possibility of early exercise
            prices[j,:] = np.where(exercise_prices[j,:] >= continuation_prices, exercise_prices[j,:], df * prices[j + 1,:])

        prices[0,:] = df * prices[1,:]
        return max(np.mean(prices[0,:]), phi * (self.S0 - K))

## Numerical Application

Let’s illustrate and compare the three pricing methods using a numerical example :

In [4]:
# Data
N = 100.
S0 = 120.
T = 0.5
vol = 0.35
r = 0.03
q = 0.01
F = S0 * math.exp((r - q) * T)

# Models Initialization
binomial = binomial_model()
binomial.set_option_data(S0, T, vol, r, q)
baw = baw_model()
baw.set_option_data(S0, T, vol, r, q)
mc = ls_mc()
mc.set_option_data(S0, T, vol, r, q)
mc.set_path()

data = {"Contract Size" : N, "Spot Price" : S0, "Maturity" : T, "ATM Implied Volatility" : vol, "ZC Rate" : r, "Dividend Yield" : q}
print("Input Data : ", data)
print("*******************************************************")
# Scenario 1 : Strike = 95% * Spot
K = 0.95 * F
print("Scenario 1  : OTM Put with Strike = 95% * Forward")
print("-------------------------------------------------------")
print("The Binomial Put Price : ", round(N * binomial.binomial_price(False, K), 2))
print("The BAW Put Price : ", round(N * baw.baw_price(False, K), 2))
print("The LS Monte-Carlo Put Price : ", round(N * mc.mc_price(False, K), 2))
print("*******************************************************")
# Scenario 2 : Strike = 105% * Spot
K = 1.05 * F
print("Scenario 2  : OTM Call with Strike = 105% * Forward")
print("-------------------------------------------------------")
print("The Binomial Call Price : ", round(N * binomial.binomial_price(True, K), 2))
print("The BAW Call Price : ", round(N * baw.baw_price(True, K), 2))
print("The LS Monte-Carlo Call Price : ", round(N * mc.mc_price(True, K), 2))

Input Data :  {'Contract Size': 100.0, 'Spot Price': 120.0, 'Maturity': 0.5, 'ATM Implied Volatility': 0.35, 'ZC Rate': 0.03, 'Dividend Yield': 0.01}
*******************************************************
Scenario 1  : OTM Put with Strike = 95% * Forward
-------------------------------------------------------
The Binomial Put Price :  879.89
The BAW Put Price :  879.62
The LS Monte-Carlo Put Price :  874.13
*******************************************************
Scenario 2  : OTM Call with Strike = 105% * Forward
-------------------------------------------------------
The Binomial Call Price :  930.0
The BAW Call Price :  930.01
The LS Monte-Carlo Call Price :  921.49
